# Algorithm Visualization and Performance Analysis

This notebook loads trained sign language recognition models, evaluates their performance on the validation dataset, 
and visualizes metrics such as accuracy, loss, and class-wise confusion matrices.

It also serves as the visualization and interpretability module for **Phase 1** of the Sign-to-Speech project.

In [1]:
# Force Auto reload for imports

%load_ext autoreload
%autoreload 2

In [2]:
# Imports

import os
import sys
import random
import subprocess
import numpy as np
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt

# Add src folder to path
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

# Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)
print("Environment initialized; reproducibility set.")

# Device check
device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    print("CUDA available:", torch.cuda.is_available())
    print(f"Using GPU: {gpu_name}")
else:
    try:
        cpu_info = subprocess.check_output(
            "lscpu | grep 'Model name'", shell=True
        ).decode().strip().split(":")[1].strip()
    except Exception:
        cpu_info = "Unknown CPU"
    print("CUDA available:", torch.cuda.is_available())
    print(f"Using CPU: {cpu_info}")


Environment initialized; reproducibility set.
CUDA available: True
Using GPU: NVIDIA GeForce RTX 4070 Ti SUPER


In [3]:
# Imports from your project
from src.dataset_loader import get_dataloaders
from src.model_definitions import make_resnet18, make_mobilenet, make_cnn_lstm
from src.train_labeled import train_model

# Load dataset
train_loader, val_loader, class_names = get_dataloaders(batch_size=32)
print(f"Loaded {len(class_names)} classes.")

# Choose model 
model_choice = "resnet18"   # change to "mobilenet" or "cnn_lstm" as needed
num_classes = len(class_names)

if model_choice == "resnet18":
    model = make_resnet18(num_classes)
elif model_choice == "mobilenet":
    model = make_mobilenet(num_classes)
else:
    model = make_cnn_lstm(num_classes)


 Dataset paths resolved:
   Project root     : /home/genesis/sem1_project/sign-to-speech-with-temporal-correction
   Train directory  : /home/genesis/sem1_project/sign-to-speech-with-temporal-correction/data/labeled/train
   Validation dir   : /home/genesis/sem1_project/sign-to-speech-with-temporal-correction/data/labeled/val

Loaded 30 classes.
Train samples: 78328, Validation samples: 8728
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space', 'val']
Loaded 30 classes.


In [4]:
# Train model
import subprocess

print("Running training script with default parameters...\n")

process = subprocess.Popen(
    ["python", "../src/train_labeled.py"],   # <-- notice the ../ here
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

for line in process.stdout:
    print(line, end="")

process.wait()
print("\nTraining script finished.")


Running training script with default parameters...

CUDA available: True
Using GPU: NVIDIA GeForce RTX 4070 Ti SUPER
 Dataset paths resolved:
   Project root     : /home/genesis/sem1_project/sign-to-speech-with-temporal-correction
   Train directory  : /home/genesis/sem1_project/sign-to-speech-with-temporal-correction/data/labeled/train
   Validation dir   : /home/genesis/sem1_project/sign-to-speech-with-temporal-correction/data/labeled/val

Loaded 30 classes.
Train samples: 78328, Validation samples: 8728
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space', 'val']

Model Selected: resnet
Training for up to 2 epochs (early stop patience = 30)


Epoch 1/2:   6%|▌         | 140/2448 [00:09<01:23, 27.79batch/s]


KeyboardInterrupt: 

In [ ]:
# --- Visualization from training_log.csv ---


log_path = "../results/training_log.csv"
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    print(df.tail())

    plt.figure(figsize=(10,5))
    plt.plot(df["epoch"], df["train_acc"], label="Train Accuracy", linewidth=2)
    plt.plot(df["epoch"], df["val_acc"], label="Validation Accuracy", linewidth=2)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.title("Training vs Validation Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(10,5))
    plt.plot(df["epoch"], df["loss"], label="Training Loss", color="red", linewidth=2)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Curve")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No training log found yet. Train the model first.")


## Summary of Evaluation

- Loaded the trained model checkpoint
- Evaluated accuracy and generated classification report
- Visualized confusion matrix and sample predictions
- (Optional) Displayed training curves from logs

Next step: integrate real-time temporal correction visualization once the Phase 2 module is ready.
